In [1]:
#explore geo data in the /mnt/storage7//jody/tb_data_2022_08_19.csv
#created - 26.08.2022

from re import sub
from unittest import result
from icecream import ic
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import re
import os
import json
import subprocess
import itertools
from statistics import mean
import pandas as pd
import chart_studio.plotly as py
import plotly.offline as po
import plotly.express as px
import plotly.graph_objs as pg
import matplotlib.pyplot as plt
import pycountry_convert
%matplotlib inline

po.init_notebook_mode(connected = True)
from urllib.request import urlopen
import pycountry
from sklearn import preprocessing

import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd

import dash
import dash_leaflet as dl
# plotly-geo
# geopandas --upgrade
# shapely
# python_version_tuple
# chart_studio

In [2]:
#load and remove nan
csv_ = "/mnt/storage7//jody/tb_data_2022_08_19.csv"
df = pd.read_csv(csv_)
df = df.dropna(subset=['country_code', 'geographic_source', 'drug_resistance_type', 'sublineage']).reset_index(drop=True)

/mnt/storage7/lwang/miniconda3/envs/ml-s7/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning:

Columns (12,15,16,17,19,20,47,48,49,51,53,54,56,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.



<h2>Creating 3 letter country code for graphing

In [3]:
#converting 2 letter country code to 3 letter
def country_code_conv(two_letter):
    name = pycountry_convert.country_alpha2_to_country_name(two_letter.upper(), cn_name_format="default")
    three_letter = pycountry_convert.country_name_to_country_alpha3(name, cn_name_format="default")

    return three_letter

df['country_3l_code'] = None
for i, e in enumerate(df['country_code']):
    df['country_3l_code'][i] =  country_code_conv(e)

/tmp/ipykernel_61718/2264284554.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
# # turning drug resistance type into number
# le = preprocessing.LabelEncoder()
# le.fit([df["drug_resistance_type"]])

# drug_resistance_type = le.transform(df["drug_resistance_type"])
# # le.inverse_transform([0, 1, 2, 3, 4, 5])


<h2>Mixed infection

In [4]:
# create a column that indicate whether the sample has mixed infection
df['mixed_infection'] = "None"

for i, x in enumerate(df['sublineage']):
    len_ = len(str(x).split(";"))
    if len_ > 1 and len_ < 3:
        df['mixed_infection'][i] = "Mixed_infection"
    else:
        df['mixed_infection'][i] = "Non-mixed_infection"


df_no_mix = df.loc[df["mixed_infection"]=="Non-mixed_infection"]
df_no_mix = df_no_mix.groupby(["country_3l_code"]).count()
df_no_mix['country_3l_code'] = df_no_mix.index

df_count = df.groupby(["country_3l_code"]).count()
df_count['country_3l_code'] = df_count.index

df_mix = df.loc[df["mixed_infection"]=="Mixed_infection"]
df_mix = df_mix.groupby(["country_3l_code"]).count()
df_mix['country_3l_code'] = df_mix.index

list_ = []
threshold = 10
for i, x in enumerate(df_count["country_3l_code"]):
    if x in df_mix["country_3l_code"]:
        mix = df_mix[df_mix['country_3l_code']==x]["mixed_infection"]
        all = df_count[df_count['country_3l_code']==x]["mixed_infection"]
        if int(all) <= threshold:
            list.append(0)
            continue
        ratio = int(mix) / int(all)
        list_.append(ratio)
    else:
        list_.append(0)

df_count["mix_rate"] = list_


/tmp/ipykernel_61718/239387552.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_61718/239387552.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df = px.data.tips()
fig = px.histogram(df_count, x="mixed_infection", nbins=200, title='Histogram of number of infection recorded',labels={'mixed_infection':'infection sample number'})
fig.show()


In [17]:
fig = px.choropleth(df_count, color='mix_rate',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<h2>Drug resistance type

In [6]:
dict_ = {}
for x in df["drug_resistance_type"].unique():
    dict_[x] = []
    df_dr = df[df["drug_resistance_type"]== x].groupby(["country_3l_code"]).count()
    df_dr["country_3l_code"] = df_dr.index
    for y in df_count["country_3l_code"]:
        if y in df_dr["country_3l_code"]:
            mix = df_dr[df_dr['country_3l_code']==y]["drug_resistance_type"]
            all = df_count[df_count['country_3l_code']==y]["drug_resistance_type"]
            dict_[x].append(int(mix)/int(all)) #if country has sample with this resistance type, insert the ratio
        else:            
            dict_[x].append(0) #else insert 0
            
for k, v in dict_.items():
    df_count[k] = v

In [24]:
fig = px.choropleth(df_count, color='MDR-TB',
                    locations='country_3l_code',
                    title="Automatic Labels Based on Data Frame Column Names"
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [26]:
fig = px.choropleth(df_count, color='Pre-XDR-TB',
                    locations='country_3l_code',
                    title="Automatic Labels Based on Data Frame Column Names"
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [29]:
fig = px.choropleth(df_count, color='Sensitive',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [30]:
fig = px.choropleth(df_count, color='Other',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [32]:
fig = px.choropleth(df_count, color='RR-TB',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [33]:
fig = px.choropleth(df_count, color='XDR-TB',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()